In [13]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64
import json

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

from CRud import Cars
pd.options.plotting.backend = "plotly"


###########################
# Data Manipulation / Model
###########################
garage = Cars()

# class read method must support return of cursor object 
df = pd.DataFrame.from_records(garage.read({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Project')

params = [
    'MPG', 'Cylinders', 'Displacment',
    'Horsepower', 'Weight', 'Acceleration', 'Model',
    'Origin'
]

app.layout = html.Div([
        
#Header
    html.Center(html.B(html.H1('Project'))),
    html.Hr(),
    
    
#new car data   
    html.H4('Edit the table below and click submit to add a new car.'),
    dt.DataTable(
        id='table-editing-simple',
        columns=(
            [{'id': 'Car', 'name': 'Car'}] +
            [{'id': p, 'name': p} for p in params]
        ),
        data=[
            dict(Car=i, **{param: 0 for param in params})
            for i in range(1)
        ],
        editable=True
    ),

#Submit Car Button
    
    html.Button('Submit New Car', id='submit-val', n_clicks=0),
    html.Div(id='container-button-basic',
             children='Enter a value and press submit'),
    html.Hr(),
    
# Dropdown1
    html.Div(
        children=[
        dcc.Dropdown(
            id="column-type",
            options=[{'label':i,'value':i} for i in df.columns],
            searchable=True            
        )]
    ),
    
# Dropdown2
    html.Div(
        children=[
        dcc.Dropdown(
            id="filter-type",
            options=[{'label':i,'value':i} for i in df.columns],
            searchable=True            
        )]
    ),
    
# Table
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        selected_columns=[],
        selected_rows=[],
        page_current=0,
        page_size=10
    ),
    html.Br(),
    html.Hr(),
    ]
)

#############################################
# Interaction Between Components / Controller
#############################################

#Dropdowns
@app.callback(Output('filter-type','options'),
              [Input('column-type', 'value')])
def update_filter(column):
    #creates new list for the second filter dropdown
    # df = pd.DataFrame.from_records(garage.read({}))
    df2 = df[column]
    ar = []
    for i in df2.keys():
        entry = df2[i]
        if entry not in ar:
            ar.append(entry)
    return([{'label':i,'value':i} for i in ar])

#Main Table
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('column-type', 'value'),
               Input('filter-type', 'value')])
def update_dashboard(column_type,filter_type):
    # New dataframe based on the filters
    # df = pd.DataFrame.from_records(garage.read({}))
    df3 = df[df[column_type].isin([filter_type])]
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df3]
    data=df3.to_dict('records')        
    return (data,columns)

#Table Highlight
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

#Submit Button
@app.callback(
    dash.dependencies.Output('container-button-basic', 'children'),
    Input('table-editing-simple', 'data'),
    [dash.dependencies.Input('submit-val', 'n_clicks')],
    )
def update_output(data, n_clicks):
    if (n_clicks >= 1):
        garage.create(data)
        n_clicks = 0
    return
    

#New Car
@app.callback(
    Output('table-editing-simple-output', 'figure'),
    Input('table-editing-simple', 'data'),
    Input('table-editing-simple', 'columns'))
def display_output(rows, columns):
    df = pd.DataFrame(rows, columns=[c['name'] for c in columns])
    return {
        'data': [{
            'type': 'parcoords',
            'dimensions': [{
                'label': col['name'],
                'values': df[col['id']]
            } for col in columns]
        }]
    }



app